# Gerando histograma normalizado

- normalizando pela quantidade de pixeis da imagem;
- normalizando pelo maior elemento do histograma;

## Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd

from zipfile import ZipFile
from skimage import io
from skimage.filters import threshold_otsu
from skimage.feature import local_binary_pattern

# Parâmetros do LBP
METHOD = 'nri_uniform'
RADIUS = 1
N_POINTS = 8

TARGET_NAMES = {'nao_faixa': 0,
                'faixa_frente': 1,
                'faixa_diagonal_direita': 2,
                'faixa_diagonal_esquerda': 3}

## Processando imagens

In [2]:
path_dict = {name: [] for name in TARGET_NAMES}

with ZipFile('./texturas.zip') as zip_file:
  for path in zip_file.namelist():
    # a pasta de cada amostra é o segundo elemento do split
    folder = path.split('/')[1]
    # Separando amostras por classe
    if folder in TARGET_NAMES:
      path_dict[folder].append(f'./texturas.zip/{path}')

# Mostrando quantidade de amostras por classe
for target in path_dict:
  print(f'{target}: {len(path_dict[target])} amostras')

amostra_n_pixels = []
amostra_max_elem = []
limiares = {name:[] for name in TARGET_NAMES}

for target_name in TARGET_NAMES:
    for path in path_dict[target_name]:
        img = io.imread(path, as_gray=True)

        # Binarizando imagem com limiar de OTSU
        img_otsu = img >= threshold_otsu(img)
        
        lbp = local_binary_pattern(image=img_otsu, P=N_POINTS, R=RADIUS, method=METHOD).flatten()
        hist = pd.Series(lbp).value_counts().sort_index()
        arr = np.zeros(59)
        for i in hist.index:
            arr[int(i)] = hist[i]

        # Normalizando pela quantidade de píxels da imagem
        norm_n_pixels = arr / arr.sum()

        df1 = pd.DataFrame(data=[norm_n_pixels])
        df1['target'] = TARGET_NAMES[target_name]
        amostra_n_pixels.append(df1)

        # Normalizando pelo maior elemento do histograma
        norm_max_elem = arr / arr.max()

        df2 = pd.DataFrame(data=[norm_max_elem])
        df2['target'] = TARGET_NAMES[target_name]
        amostra_max_elem.append(df2)


data_n_pixels = pd.concat(amostra_n_pixels, ignore_index=True)
data_n_pixels.to_csv('./csvs/otsu-norm_n_pixels-lbp_faixas.csv')

data_max_elem = pd.concat(amostra_max_elem, ignore_index=True)
data_max_elem.to_csv('./csvs/otsu-norm_max_elem-lbp_faixas.csv')

nao_faixa: 750 amostras
faixa_frente: 250 amostras
faixa_diagonal_direita: 250 amostras
faixa_diagonal_esquerda: 250 amostras
